## `Inflaition Data Set: 2005-2023`

In [8]:
import pandas as pd 
import matplotlib.pyplot as plt
import smplotlib

inflation = pd.read_csv("/Users/aleskamedrano/Real_Estate_SeminarFall2025/inflation_all_regions.csv")
#print (inflation)
# Rename columns:
infl = inflation.rename(columns={"country": "region"})

#print(infl.columns)

#annual inflation
infl_dec = infl[infl["month"] == 12].copy()
infl_dec["annual_inflation"] = infl_dec.groupby("region")["value"].pct_change()

infl_annual = infl_dec.dropna(subset=["annual_inflation"])[["region", "year", "annual_inflation"]]
infl_annual = infl_annual.rename(columns={"region": "region_name", "year": "infl_year"})

#print(infl_annual)
#save the new inflation data set
infl_annual.to_csv("inflation_final_dataset.csv", index=False)

`Final Inflaition Dataset`

In [10]:
inflation = pd.read_csv("/Users/aleskamedrano/Real_Estate_SeminarFall2025/inflation_final_dataset.csv")
print(inflation)

   region_name  infl_year  annual_inflation
0    Northeast       2005          0.035166
1    Northeast       2006          0.029665
2    Northeast       2007          0.038220
3    Northeast       2008          0.007457
4    Northeast       2009          0.028304
..         ...        ...               ...
71       South       2019          0.021222
72       South       2020          0.013765
73       South       2021          0.074075
74       South       2022          0.070348
75       South       2023          0.036602

[76 rows x 3 columns]


## `ACS Data Set`

In [11]:
import pyarrow.csv as csv
import pyarrow.parquet as pq

# Load data into parquet so pandas doesn't crush: So instead of loading into RAM it loads by columns
table = csv.read_csv("/Users/aleskamedrano/Real_Estate_SeminarFall2025/ACS_vf_2005-2023.csv.gz")
pq.write_table(table, "ACS_2005_2023.parquet")

#Use pyarrow to analyze 

In [13]:
import pyarrow.dataset as ds

acs_dataset = ds.dataset("/Users/aleskamedrano/Real_Estate_SeminarFall2025/ACS_2005_2023.parquet")
print(acs_dataset.schema)



YEAR: int64
SAMPLE: int64
SERIAL: int64
CBSERIAL: int64
HHWT: double
CLUSTER: int64
REGION: int64
STATEFIP: int64
STRATA: int64
GQ: int64
HHINCOME: int64
PERNUM: int64
PERWT: double
AGE: int64
MARST: int64
EDUC: int64
EDUCD: int64
EMPSTAT: int64
EMPSTATD: int64


`Load only the control variables to the regression`

In [3]:
import pyarrow.dataset as ds
import pandas as pd

# Load the dataset (lazy loading)
dataset = ds.dataset("/Users/aleskamedrano/Real_Estate_SeminarFall2025/ACS_2005_2023.parquet")

# Columns of interest
cols = [
    "YEAR", "REGION", "AGE", "HHINCOME",
    "EDUC", "MARST", "EMPSTAT"
]

# Load only selected columns
table = dataset.to_table(columns=cols)

# Convert to pandas
df = table.to_pandas()

# Inspect data
print(df.head(50))
print(df.shape)
#print(df.describe())
#print(df.isna().sum())


    YEAR  REGION  AGE  HHINCOME  EDUC  MARST  EMPSTAT
0   2004      32   52     21380     7      4        1
1   2004      32   31     21380    10      6        1
2   2004      32   14     21380     2      6        0
3   2004      32    6     21380     1      6        0
4   2004      32   51    122000     7      1        1
5   2004      32   51    122000    11      1        1
6   2004      32   13    122000     2      6        0
7   2004      32   48     99000     6      1        1
8   2004      32   51     99000     6      1        1
9   2004      32   24     99000     7      4        1
10  2004      32   20     99000     7      6        1
11  2004      32   67     21900     8      5        1
12  2004      32   51     37100     6      1        3
13  2004      32   55     37100     6      1        3
14  2004      32   27     36000     7      1        1
15  2004      32   25     36000     7      1        1
16  2004      32    5     36000     1      6        0
17  2004      32    3     36

## `Inflaition Experience Construction`

`Load both data sets`

In [19]:
import pandas as pd

#df is the acs data set in pandas with only 8 selecetd variables
acs = df.copy()  
print("ACS HEAD:\n", acs.head())
print(acs["REGION"].unique())

infl = pd.read_csv("/Users/aleskamedrano/Real_Estate_SeminarFall2025/inflation_final_dataset.csv") 
print("INFLATION DATA HEAD:\n", infl.head())

# Make sure column names match ACS
#infl = infl.rename(columns={
   # "region_name": "REGION"
#})

#print("Renamed inflation columns:\n", infl.columns)



ACS HEAD:
    YEAR  REGION  AGE  HHINCOME  EDUC  MARST  EMPSTAT
0  2004      32   52     21380     7      4        1
1  2004      32   31     21380    10      6        1
2  2004      32   14     21380     2      6        0
3  2004      32    6     21380     1      6        0
4  2004      32   51    122000     7      1        1
[32 42 41 33 11 31 21 22 12]
INFLATION DATA HEAD:
   region_name  infl_year  annual_inflation
0   Northeast       2005          0.035166
1   Northeast       2006          0.029665
2   Northeast       2007          0.038220
3   Northeast       2008          0.007457
4   Northeast       2009          0.028304


In [16]:
print(acs["REGION"].head(20))
print(acs["REGION"].unique())
print(acs["REGION"].value_counts())



0     32
1     32
2     32
3     32
4     32
5     32
6     32
7     32
8     32
9     32
10    32
11    32
12    32
13    32
14    32
15    32
16    32
17    32
18    32
19    32
Name: REGION, dtype: int64
[32 42 41 33 11 31 21 22 12]
REGION
31    11708103
42     9593763
21     9080544
12     7902052
33     6949817
41     4397394
22     4140879
32     3607670
11     2860908
Name: count, dtype: int64


`Compute each persons birth year`
- birth_year=YEAR−AGE

In [27]:
acs["birth_year"] = acs["YEAR"] - acs["AGE"]
print("After adding birth_year:\n", acs[["YEAR","AGE","birth_year"]].head())

After adding birth_year:
    YEAR  AGE  birth_year
0  2004   52        1952
1  2004   31        1973
2  2004   14        1990
3  2004    6        1998
4  2004   51        1953


`Merge ACS with inflation data by region`

You need the inflation data that matches the person’s region:
- Northeast
- Midwest
- South
- West
Merge on:
REGION (ACS) ↔ region_name (inflation set)


In [28]:
region_map = {
    11: "Northeast",
    12: "Northeast",
    21: "Midwest",
    22: "Midwest",
    31: "South",
    32: "South",
    33: "South",
    41: "West",
    42: "West"
}

#Apply to my ACS data:
acs["REGION"] = pd.to_numeric(acs["REGION"], errors="coerce")


In [29]:
#rename the column to match th einflaiiton data set:
acs["region_name"] = acs["REGION"].map(region_map)


In [30]:
print(acs["region_name"].value_counts())
print(acs["region_name"].tail(10))

region_name
South        22265590
West         13991157
Midwest      13221423
Northeast    10762960
Name: count, dtype: int64
60241120    West
60241121    West
60241122    West
60241123    West
60241124    West
60241125    West
60241126    West
60241127    West
60241128    West
60241129    West
Name: region_name, dtype: object


In [31]:
print(acs.columns.tolist())


['YEAR', 'REGION', 'AGE', 'HHINCOME', 'EDUC', 'MARST', 'EMPSTAT', 'region_name', 'birth_year']


In [32]:
#Saving ACS dataset with all control variables that I can use later
acs.to_parquet("/Users/aleskamedrano/Real_Estate_SeminarFall2025/ACS_cleaned_2005_2023.parquet",
               index=False)


In [ ]:
#print("Saved. Shape:", acs.shape)


Saved. Shape: (60241130, 9)


`Merging`

In [4]:
import pandas as pd

# Load ACS (parquet)
acs = pd.read_parquet("/Users/aleskamedrano/Real_Estate_SeminarFall2025/ACS_cleaned_2005_2023.parquet")

# Load inflation (csv)
infl = pd.read_csv("/Users/aleskamedrano/Real_Estate_SeminarFall2025/inflation_final_dataset.csv")

print("ACS sample:\n", acs.head())
print("\nInflation sample:\n", infl.head())


ACS sample:
    YEAR  REGION  AGE  HHINCOME  EDUC  MARST  EMPSTAT region_name  birth_year
0  2004      32   52     21380     7      4        1       South        1952
1  2004      32   31     21380    10      6        1       South        1973
2  2004      32   14     21380     2      6        0       South        1990
3  2004      32    6     21380     1      6        0       South        1998
4  2004      32   51    122000     7      1        1       South        1953

Inflation sample:
   region_name  infl_year  annual_inflation
0   Northeast       2005          0.035166
1   Northeast       2006          0.029665
2   Northeast       2007          0.038220
3   Northeast       2008          0.007457
4   Northeast       2009          0.028304


`Divide and process into batches`

In [5]:
BATCH_SIZE = 50_000


In [6]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

output_path = "/Users/aleskamedrano/Real_Estate_SeminarFall2025/ACS_expanded/"


In [7]:
import os

# Ensure folder exists
os.makedirs(output_path, exist_ok=True)

BATCH_SIZE = 50_000
num_rows = len(acs)

print("Total ACS rows:", num_rows)

batch_id = 0

for start in range(0, num_rows, BATCH_SIZE):
    end = min(start + BATCH_SIZE, num_rows)
    batch = acs.iloc[start:end]

    print(f"\nProcessing batch {batch_id} | Rows {start} to {end}")

    rows = []

    for idx, row in batch.iterrows():
        birth = int(row["birth_year"])
        survey_year = int(row["YEAR"])

        # Experience years = from birth to year before survey
        years = list(range(birth, survey_year))

        for infl_yr in years:
            rows.append({
                "region_name": row["region_name"],
                "YEAR": survey_year,
                "AGE": row["AGE"],
                "HHINCOME": row["HHINCOME"],
                "EDUC": row["EDUC"],
                "MARST": row["MARST"],
                "EMPSTAT": row["EMPSTAT"],
                "birth_year": birth,
                "infl_year": infl_yr
            })

    # Convert this batch to a table
    batch_df = pd.DataFrame(rows)
    batch_table = pa.Table.from_pandas(batch_df)

    # Save batch
    pq.write_table(batch_table, f"{output_path}/exp_batch_{batch_id}.parquet")

    print(f"Saved batch {batch_id}, shape = {batch_df.shape}")
    batch_id += 1

print("\nAll batches processed!")


Total ACS rows: 60241130

Processing batch 0 | Rows 0 to 50000
Saved batch 0, shape = (1921634, 9)

Processing batch 1 | Rows 50000 to 100000
Saved batch 1, shape = (1863342, 9)

Processing batch 2 | Rows 100000 to 150000
Saved batch 2, shape = (1849319, 9)

Processing batch 3 | Rows 150000 to 200000
Saved batch 3, shape = (1880640, 9)

Processing batch 4 | Rows 200000 to 250000
Saved batch 4, shape = (2049997, 9)

Processing batch 5 | Rows 250000 to 300000
Saved batch 5, shape = (1978915, 9)

Processing batch 6 | Rows 300000 to 350000
Saved batch 6, shape = (1884583, 9)

Processing batch 7 | Rows 350000 to 400000
Saved batch 7, shape = (1877380, 9)

Processing batch 8 | Rows 400000 to 450000
Saved batch 8, shape = (1955702, 9)

Processing batch 9 | Rows 450000 to 500000
Saved batch 9, shape = (1920834, 9)

Processing batch 10 | Rows 500000 to 550000
Saved batch 10, shape = (1920471, 9)

Processing batch 11 | Rows 550000 to 600000
Saved batch 11, shape = (1886511, 9)

Processing batch 

In [8]:
import pandas as pd
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import pyarrow as pa
import os
from tqdm import tqdm

expanded_path = "/Users/aleskamedrano/Real_Estate_SeminarFall2025/ACS_expanded/"
infl_path = "/Users/aleskamedrano/Real_Estate_SeminarFall2025/inflation_final_dataset.csv"
output_path = "/Users/aleskamedrano/Real_Estate_SeminarFall2025/InflationExperience_Batches/"

os.makedirs(output_path, exist_ok=True)


In [9]:
infl = pd.read_csv(infl_path)
print("Inflation data preview:", infl.head())


Inflation data preview:   region_name  infl_year  annual_inflation
0   Northeast       2005          0.035166
1   Northeast       2006          0.029665
2   Northeast       2007          0.038220
3   Northeast       2008          0.007457
4   Northeast       2009          0.028304


In [10]:
batch_files = sorted(os.listdir(expanded_path))
batch_files = [f for f in batch_files if f.endswith(".parquet")]

print("Number of expanded batches:", len(batch_files))

batch_id = 0

for file in tqdm(batch_files):
    print(f"\nProcessing inflation for batch: {file}")

    # Load small batch
    batch = pq.read_table(expanded_path + file).to_pandas()
    
    # Merge with inflation on region + year
    merged = batch.merge(
        infl,
        how="left",
        on=["region_name", "infl_year"]
    )

    # Compute linear weights
    merged["k"] = merged.groupby(["YEAR", "birth_year"]).cumcount()
    merged["weight"] = merged["AGE"] - merged["k"]
    merged["weighted_infl"] = merged["annual_inflation"] * merged["weight"]

    # Collapse to inflation experience for that batch
    infl_exp_batch = merged.groupby(
        ["YEAR", "birth_year", "region_name"]
    ).apply(
        lambda x: x["weighted_infl"].sum() / x["weight"].sum()
    ).reset_index(name="inflation_experience")

    # Save collapsed batch — tiny file!
    pq.write_table(
        pa.Table.from_pandas(infl_exp_batch),
        f"{output_path}/infl_exp_batch_{batch_id}.parquet"
    )

    print(f"Saved infl_exp_batch_{batch_id}.parquet | rows = {len(infl_exp_batch)}")
    
    batch_id += 1

print("\nAll inflation experience batches processed!")


Number of expanded batches: 1205


  0%|          | 0/1205 [00:00<?, ?it/s]


Processing inflation for batch: exp_batch_0.parquet


/var/folders/ws/jw7vmdhx3yq2zps6fh4r4wmc0000gn/T/ipykernel_77186/2992111468.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(
  0%|          | 1/1205 [00:00<11:32,  1.74it/s]

Saved infl_exp_batch_0.parquet | rows = 178

Processing inflation for batch: exp_batch_1.parquet


...

Saved infl_exp_batch_1202.parquet | rows = 89

Processing inflation for batch: exp_batch_998.parquet


/var/folders/ws/jw7vmdhx3yq2zps6fh4r4wmc0000gn/T/ipykernel_77186/2992111468.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(
100%|█████████▉| 1204/1205 [09:36<00:00,  2.10it/s]

Saved infl_exp_batch_1203.parquet | rows = 89

Processing inflation for batch: exp_batch_999.parquet


/var/folders/ws/jw7vmdhx3yq2zps6fh4r4wmc0000gn/T/ipykernel_77186/2992111468.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(
100%|██████████| 1205/1205 [09:37<00:00,  2.09it/s]

Saved infl_exp_batch_1204.parquet | rows = 271

All inflation experience batches processed!


In [2]:
import pyarrow.dataset as ds
import pandas as pd

infl_exp_path = "/Users/aleskamedrano/Real_Estate_SeminarFall2025/InflationExperience_Batches/"

# Combine all collapsed inflation experience files
infl_exp_dataset = ds.dataset(infl_exp_path, format="parquet")
infl_exp_table = infl_exp_dataset.to_table()
infl_exp_final = infl_exp_table.to_pandas()

print("Combined inflation experience shape:", infl_exp_final.shape)
print(infl_exp_final.head())


Combined inflation experience shape: (164757, 4)
   YEAR  birth_year region_name  inflation_experience
0  2004        1912       South                  -0.0
1  2004        1913       South                  -0.0
2  2004        1914        West                  -0.0
3  2004        1916       South                  -0.0
4  2004        1917       South                  -0.0


`Filter inflation years to each person's lifespan`

A person born in 1990 and surveyed in 2020 should only get inflation years:
- 1990 → 2019 (the year before survey)

Must:
- Keep only inflation years ≥ birth_year

- Keep only inflation years ≤ YEAR − 1

In [3]:
infl_exp_final_filtered = infl_exp_final[infl_exp_final["birth_year"] >= 2004].copy()
print(infl_exp_final_filtered.head())
print("Shape:", infl_exp_final_filtered.shape)


     YEAR  birth_year region_name  inflation_experience
501  2021        2004       South              0.018658
502  2021        2004        West              0.021081
503  2021        2005       South              0.019823
504  2021        2005        West              0.022396
505  2021        2006       South              0.018672
Shape: (17783, 4)


In [4]:
import pandas as pd

acs = pd.read_parquet("/Users/aleskamedrano/Real_Estate_SeminarFall2025/ACS_cleaned_2005_2023.parquet")
print(acs.head())


   YEAR  REGION  AGE  HHINCOME  EDUC  MARST  EMPSTAT region_name  birth_year
0  2004      32   52     21380     7      4        1       South        1952
1  2004      32   31     21380    10      6        1       South        1973
2  2004      32   14     21380     2      6        0       South        1990
3  2004      32    6     21380     1      6        0       South        1998
4  2004      32   51    122000     7      1        1       South        1953


In [5]:
acs_filtered = acs[acs["birth_year"] >= 2004].copy()
print("ACS filtered:", acs_filtered.shape)




ACS filtered: (7322566, 9)


In [6]:
acs_filtered.head(10)

,YEAR,REGION,AGE,HHINCOME,EDUC,MARST,EMPSTAT,region_name,birth_year
145,2004,32,0,115900,0,6,0,South,2004
209,2004,32,0,100000,0,6,0,South,2004
321,2004,32,0,71300,0,6,0,South,2004
398,2004,32,0,40300,0,6,0,South,2004
424,2004,32,0,40000,0,6,0,South,2004
629,2004,32,0,3340,0,6,0,South,2004
630,2004,32,0,3340,0,6,0,South,2004
656,2004,32,0,0,0,6,0,South,2004
687,2004,32,0,60900,0,6,0,South,2004
782,2004,32,0,28000,0,6,0,South,2004


In [10]:
import pyarrow.dataset as ds
import pandas as pd

# Path to inflation experience batches
infl_exp_path = "/Users/aleskamedrano/Real_Estate_SeminarFall2025/InflationExperience_Batches/"

# Load as a pyarrow dataset
infl_ds = ds.dataset(infl_exp_path, format="parquet")

# Convert to pandas in one efficient step
infl_exp_final = infl_ds.to_table().to_pandas()

print("Combined inflation experience shape:", infl_exp_final.shape)
print(infl_exp_final.head())


Combined inflation experience shape: (164757, 4)
   YEAR  birth_year region_name  inflation_experience
0  2004        1912       South                  -0.0
1  2004        1913       South                  -0.0
2  2004        1914        West                  -0.0
3  2004        1916       South                  -0.0
4  2004        1917       South                  -0.0


In [11]:
infl_exp_final_filtered = infl_exp_final[infl_exp_final["birth_year"] >= 2004].copy()

print("Filtered shape:", infl_exp_final_filtered.shape)
print(infl_exp_final_filtered.head())


Filtered shape: (17783, 4)
     YEAR  birth_year region_name  inflation_experience
501  2021        2004       South              0.018658
502  2021        2004        West              0.021081
503  2021        2005       South              0.019823
504  2021        2005        West              0.022396
505  2021        2006       South              0.018672


In [12]:
infl_exp_final_filtered.to_parquet(
    "/Users/aleskamedrano/Real_Estate_SeminarFall2025/infl_exp_final_filtered.parquet"
)


- `Merge by chunks`

In [13]:
import pandas as pd

infl = pd.read_parquet("/Users/aleskamedrano/Real_Estate_SeminarFall2025/infl_exp_final_filtered.parquet")

print("Inflation table shape:", infl.shape)



Inflation table shape: (17783, 4)


In [16]:
import os

merged_folder = "/Users/aleskamedrano/Real_Estate_SeminarFall2025/Merged_Batches/"
os.makedirs(merged_folder, exist_ok=True)


In [ ]:
import pyarrow.dataset as ds
import pandas as pd
from tqdm import tqdm
import os

# ---- PATHS ----
acs_path = "/Users/aleskamedrano/Real_Estate_SeminarFall2025/ACS_cleaned_2005_2023.parquet"
infl_path = "/Users/aleskamedrano/Real_Estate_SeminarFall2025/infl_exp_final_filtered.parquet"
merged_folder = "/Users/aleskamedrano/Real_Estate_SeminarFall2025/Merged_Batches/"

# ---- LOAD SMALL INFLATION FILE ----
infl = pd.read_parquet(infl_path)
print("Inflation data shape:", infl.shape)

# ---- LOAD ACS AS DATASET ----
acs_ds = ds.dataset(acs_path, format="parquet")

# Columns needed
acs_columns = ["YEAR", "AGE", "HHINCOME", "EDUC", "MARST",
               "EMPSTAT", "region_name", "birth_year"]

# ---- CREATE SCANNER ----
scanner = ds.Scanner.from_dataset(
    acs_ds,
    columns=acs_columns,
    batch_size=200_000
)

print("\nStarting streaming merge...\n")

batch_id = 0

for batch in tqdm(scanner.to_batches()):
    acs_chunk = batch.to_pandas()

    # Filter
    acs_chunk = acs_chunk[acs_chunk["birth_year"] >= 2004]

    # Merge with inflation experience
    merged_chunk = acs_chunk.merge(
        infl,
        how="left",
        on=["YEAR", "birth_year", "region_name"]
    )

    # Write each batch to its own file
    out_path = os.path.join(merged_folder, f"merged_batch_{batch_id}.parquet")
    merged_chunk.to_parquet(out_path)

    batch_id += 1

print(f"\nDONE! {batch_id} merged batches created in {merged_folder}")


Inflation data shape: (17783, 4)

Starting streaming merge...



345it [00:26, 12.84it/s]


DONE! 345 merged batches created in /Users/aleskamedrano/Real_Estate_SeminarFall2025/Merged_Batches/


In [1]:
import pyarrow.dataset as ds
import pyarrow as pa

merged_folder = "/Users/aleskamedrano/Real_Estate_SeminarFall2025/Merged_Batches/"
output_path = "/Users/aleskamedrano/Real_Estate_SeminarFall2025/ACS_merged_streamed/"

# Load all merged parquet batches as a dataset
merged_ds = ds.dataset(merged_folder, format="parquet")

# Write them out as ONE unified dataset
ds.write_dataset(
    data=merged_ds,
    base_dir=output_path,
    format="parquet",
    # Combine files:
    partitioning=None,
    existing_data_behavior="overwrite_or_ignore"
)

print("Final merged dataset written to:", output_path)


Final merged dataset written to: /Users/aleskamedrano/Real_Estate_SeminarFall2025/ACS_merged_streamed/


In [1]:
import pandas as pd
acs_main = pd.read_parquet("/Users/aleskamedrano/Real_Estate_SeminarFall2025/ACS_cleaned_2005_2023.parquet")


In [2]:
import pandas as pd

owners = pd.read_csv(
    "/Users/aleskamedrano/Real_Estate_SeminarFall2025/ownership_2005-2023.csv.gz",
    low_memory=False   
)


KeyboardInterrupt: 

## `Regression`

In [24]:
import pyarrow.dataset as ds
import pyarrow as pa
import pandas as pd

acs_full = ds.dataset("/Users/aleskamedrano/Real_Estate_SeminarFall2025/ACS_2005_2023.parquet")

cols_needed = [
    "YEAR", "REGION", "AGE", "HHINCOME",
    "EDUC", "MARST", "EMPSTAT", "OWNERSHP"
]

scanner = acs_full.scanner(columns=cols_needed, batch_size=250000)

df_list = []
for batch in scanner.to_batches():
    df_list.append(batch.to_pandas())

acs_clean = pd.concat(df_list, ignore_index=True)

# Add region_name + birth_year
region_map = {11:'Northeast',21:'Midwest',31:'South',41:'West',
              12:'Northeast',22:'Midwest',32:'South',42:'West',
              13:'Northeast',23:'Midwest',33:'South',43:'West'}

acs_clean["region_name"] = acs_clean["REGION"].map(region_map)
acs_clean["birth_year"] = acs_clean["YEAR"] - acs_clean["AGE"]

acs_clean.to_parquet("/Users/aleskamedrano/Real_Estate_SeminarFall2025/ACS_cleaned_2005_2023.parquet")

print("NEW ACS_cleaned created:", acs_clean.shape)


ArrowInvalid: No match for FieldRef.Name(OWNERSHP) in YEAR: int64
SAMPLE: int64
SERIAL: int64
CBSERIAL: int64
HHWT: double
CLUSTER: int64
REGION: int64
STATEFIP: int64
STRATA: int64
GQ: int64
HHINCOME: int64
PERNUM: int64
PERWT: double
AGE: int64
MARST: int64
EDUC: int64
EDUCD: int64
EMPSTAT: int64
EMPSTATD: int64
__fragment_index: int32
__batch_index: int32
__last_in_fragment: bool
__filename: string

In [25]:
ds.dataset("/Users/aleskamedrano/Real_Estate_SeminarFall2025/ACS_cleaned_2005_2023.parquet").schema


YEAR: int64
REGION: int64
AGE: int64
HHINCOME: int64
EDUC: int64
MARST: int64
EMPSTAT: int64
region_name: string
birth_year: int64
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1067